# Forecasting the Next Year's Points Per Game Average...
## Part 0 of building an ensemble Temporal Fusion Transformer for predicting a player's retirement

In this notebook I attempt a simplified LSTM/TFT model to develop an understanding of how to pre-process and feed my data into a time series model. Using ChatGPT's suggestions for intial code structure and dummy data, I refine the my own method for handling training data. There are likely plenty of logical fallacies in certain decisions made with my data, but for the sake of simplicty and a working/trained model as an end result, the following shall suffice.

The following steps were taken to reach a trainable final result.
1. Source Model Architecture -- I had initally hoped to develop something a bit more complex, but there were too many bugs to reasonably assume this task would be completed. This is notebook 5 of 5 in these trials.
2. Scrape and Clean Data -- I scraped season stats of all players over the past 25 seasons. Each entry contained 71 columns, I focused on the main stats like points/rebounds/assists per game. Because there are players within this set that either have stats before the 25 season window (i.e. played in seasons prior to the 1999-00 season) as well as rookie or early career players, I only included players with a season count greater than 5. Then I took the five most recent seasons of those players. The number 5 is somewhat arbitrary but was chosen to mimic the choice of chatGPT's suggested model.
3. Split, Feed, Train -- I split the seasons such that the first 4 (of each player) would be the training data, while the last season would be the target. The data types/objects were then converted to shapes accepted by the tensors. The model was then trained. And gave predictions for our target.
4. Evaluate --  Not all steps of evaluation were taken. It is apparent from the predictions that our model can be accurate but often misses the mark (sometimes by an order of magnitude). The final evaluation shown takes the predicted values and divides by the target. The closer a number is to 1, the closer to 100% accurate that prediciton is (each prediction correspond to a player and there previous 5 seasons or final 5 in the case of inactive players.

In [13]:
import pandas as pd

In [1]:
import torch
import torch.nn as nn

# Example data for 2 players over 5 seasons
# Features: [Points per Game (PPG), Age, Games Played]
x_temporal = torch.tensor([
    # Player 1
    [[20, 23, 75],   # Season 1: 20 PPG, 23 years old, 75 games played
     [22, 24, 80],   # Season 2: 22 PPG, 24 years old, 80 games played
     [24, 25, 78],   # Season 3: 24 PPG, 25 years old, 78 games played
     [21, 26, 65],   # Season 4: 21 PPG, 26 years old, 65 games played
     [23, 27, 70]],  # Season 5: 23 PPG, 27 years old, 70 games played
    
    # Player 2
    [[15, 22, 72],   # Season 1: 15 PPG, 22 years old, 72 games played
     [18, 23, 77],   # Season 2: 18 PPG, 23 years old, 77 games played
     [16, 24, 74],   # Season 3: 16 PPG, 24 years old, 74 games played
     [19, 25, 68],   # Season 4: 19 PPG, 25 years old, 68 games played
     [17, 26, 71]]   # Season 5: 17 PPG, 26 years old, 71 games played
], dtype=torch.float32)

# Target: Predict next season's Points per Game (PPG) for each player
y_target = torch.tensor([
    [24],  # Player 1: Target is next season's PPG (24)
    [20]   # Player 2: Target is next season's PPG (20)
], dtype=torch.float32)

# Step 1: Define a simple TFT-like model (this is not the full TFT architecture but a simplified version)

class SimpleTFT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.1):
        super(SimpleTFT, self).__init__()
        
        # LSTM to handle the temporal nature of the data (sequences over time)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to predict the next PPG
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        # Activation function for the output
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Pass the input through the LSTM
        lstm_out, _ = self.lstm(x)
        
        # Take the last time step's output (since LSTM outputs a sequence)
        last_timestep = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_dim)
        
        # Pass it through a fully connected layer and apply ReLU
        output = self.fc(last_timestep)
        return self.relu(output)

# Step 2: Hyperparameters
input_dim = 3    # We have 3 features per season: PPG, Age, Games Played
hidden_dim = 16  # Number of hidden units in the LSTM
output_dim = 1   # Predicting a single value: next season's PPG
num_layers = 2   # Number of LSTM layers

# Create an instance of the model
model = SimpleTFT(input_dim, hidden_dim, output_dim, num_layers)

# Step 3: Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 4: Training loop
num_epochs = 100  # Number of epochs to train

for epoch in range(num_epochs):
    # Zero gradients
    optimizer.zero_grad()
    
    # Forward pass: Pass the temporal data through the model
    output = model(x_temporal)
    
    # Compute the loss (difference between predicted and actual PPG)
    loss = criterion(output, y_target)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 5: Make predictions
with torch.no_grad():
    predictions = model(x_temporal)
    print("Predicted PPG for next season:")
    print(predictions)



Epoch [10/100], Loss: 488.0000
Epoch [20/100], Loss: 488.0000
Epoch [30/100], Loss: 488.0000
Epoch [40/100], Loss: 488.0000
Epoch [50/100], Loss: 488.0000
Epoch [60/100], Loss: 488.0000
Epoch [70/100], Loss: 488.0000
Epoch [80/100], Loss: 488.0000
Epoch [90/100], Loss: 488.0000
Epoch [100/100], Loss: 488.0000
Predicted PPG for next season:
tensor([[0.],
        [0.]])


In [3]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

# Example data for 2 players over 5 seasons
# Features: [Points per Game (PPG), Age, Games Played]
x_temporal = torch.tensor([
    # Player 1
    [[20, 23, 75],   # Season 1: 20 PPG, 23 years old, 75 games played
     [22, 24, 80],   # Season 2: 22 PPG, 24 years old, 80 games played
     [24, 25, 78],   # Season 3: 24 PPG, 25 years old, 78 games played
     [21, 26, 65],   # Season 4: 21 PPG, 26 years old, 65 games played
     [23, 27, 70]],  # Season 5: 23 PPG, 27 years old, 70 games played
    
    # Player 2
    [[15, 22, 72],   # Season 1: 15 PPG, 22 years old, 72 games played
     [18, 23, 77],   # Season 2: 18 PPG, 23 years old, 77 games played
     [16, 24, 74],   # Season 3: 16 PPG, 24 years old, 74 games played
     [19, 25, 68],   # Season 4: 19 PPG, 25 years old, 68 games played
     [17, 26, 71]]   # Season 5: 17 PPG, 26 years old, 71 games played
], dtype=torch.float32)

# Target: Predict next season's Points per Game (PPG) for each player
y_target = torch.tensor([
    [24],  # Player 1: Target is next season's PPG (24)
    [20]   # Player 2: Target is next season's PPG (20)
], dtype=torch.float32)

# Scale the input data
scaler = MinMaxScaler()

# Reshape to 2D for scaling
x_reshaped = x_temporal.view(-1, 3).numpy()
x_scaled = scaler.fit_transform(x_reshaped)

# Reshape back to original shape
x_temporal_scaled = torch.tensor(x_scaled).view(x_temporal.shape).float()

# Model definition (Updated: No ReLU in final layer)
class SimpleTFT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.1):
        super(SimpleTFT, self).__init__()
        
        # LSTM to handle the temporal nature of the data (sequences over time)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to predict the next PPG
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Pass the input through the LSTM
        lstm_out, _ = self.lstm(x)
        
        # Take the last time step's output (since LSTM outputs a sequence)
        last_timestep = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_dim)
        
        # Pass it through a fully connected layer
        output = self.fc(last_timestep)
        return output

# Hyperparameters
input_dim = 3    # We have 3 features per season: PPG, Age, Games Played
hidden_dim = 16  # Number of hidden units in the LSTM
output_dim = 1   # Predicting a single value: next season's PPG
num_layers = 2   # Number of LSTM layers

# Create an instance of the model
model = SimpleTFT(input_dim, hidden_dim, output_dim, num_layers)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100  # Number of epochs to train

for epoch in range(num_epochs):
    # Zero gradients
    optimizer.zero_grad()
    
    # Forward pass: Pass the temporal data through the model
    output = model(x_temporal_scaled)
    
    # Compute the loss (difference between predicted and actual PPG)
    loss = criterion(output, y_target)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predictions = model(x_temporal_scaled)
    print("Predicted PPG for next season:")
    print(predictions)



Epoch [10/100], Loss: 486.3284
Epoch [20/100], Loss: 482.6341
Epoch [30/100], Loss: 477.2782
Epoch [40/100], Loss: 469.8988
Epoch [50/100], Loss: 457.9529
Epoch [60/100], Loss: 441.7460
Epoch [70/100], Loss: 419.5346
Epoch [80/100], Loss: 397.8803
Epoch [90/100], Loss: 377.6301
Epoch [100/100], Loss: 360.5851
Predicted PPG for next season:
tensor([[3.1209],
        [3.1316]])


In [7]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

# Example data for 2 players over 5 seasons
x_temporal = torch.tensor([
    # Player 1
    [[20, 23, 75],   
     [22, 24, 80],   
     [24, 25, 78],   
     [21, 26, 65],   
     [23, 27, 70]],  
    
    # Player 2
    [[15, 22, 72],   
     [18, 23, 77],   
     [16, 24, 74],   
     [19, 25, 68],   
     [17, 26, 71]]  
], dtype=torch.float32)

# Target: Predict next season's PPG
y_target = torch.tensor([
    [24],  
    [20]   
], dtype=torch.float32)

# Scale the input data
scaler = MinMaxScaler()

# Reshape to 2D for scaling
x_reshaped = x_temporal.view(-1, 3).numpy()
x_scaled = scaler.fit_transform(x_reshaped)

# Reshape back to original shape
x_temporal_scaled = torch.tensor(x_scaled).view(x_temporal.shape).float()

# Scale the target (PPG values)
y_target_scaled = scaler.fit_transform(y_target)

# Model definition (Updated: No ReLU in final layer)
class SimpleTFT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.1):
        super(SimpleTFT, self).__init__()
        
        # LSTM to handle the temporal nature of the data
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to predict next PPG
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)
        
        # Take last timestep's output
        last_timestep = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_dim)
        
        # Fully connected output
        output = self.fc(last_timestep)
        return output

# Hyperparameters
input_dim = 3    # 3 features: PPG, Age, Games Played
hidden_dim = 16  # Number of hidden units
output_dim = 1   # Predicting 1 value: next season's PPG
num_layers = 2   # Number of LSTM layers

# Create the model
model = SimpleTFT(input_dim, hidden_dim, output_dim, num_layers)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adjusted learning rate

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero gradients
    
    # Forward pass
    output = model(x_temporal_scaled)
    
    # Compute loss
    loss = criterion(output, torch.tensor(y_target_scaled, dtype=torch.float32))
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predictions_scaled = model(x_temporal_scaled)
    
    # Inverse scale predictions back to original range
    predictions = scaler.inverse_transform(predictions_scaled)
    
    print("Predicted PPG for next season:")
    print(predictions)


Epoch [10/100], Loss: 0.2468
Epoch [20/100], Loss: 0.1943
Epoch [30/100], Loss: 0.0127
Epoch [40/100], Loss: 0.0142
Epoch [50/100], Loss: 0.0007
Epoch [60/100], Loss: 0.0023
Epoch [70/100], Loss: 0.0030
Epoch [80/100], Loss: 0.0018
Epoch [90/100], Loss: 0.0005
Epoch [100/100], Loss: 0.0014
Predicted PPG for next season:
[[23.87578559]
 [19.99205014]]


In [33]:
data = pd.read_csv("nba_sznAvg_last25.csv")

# Ensure 'SEASON' is treated as the season index
# data['SEASON'] = data['SEASON'].astype(int)

# Filter data from season 5 onwards
# data = data[data['SEASON'] >= 5].copy()

# Calculate averages
data['MPG'] = data['MIN'] / data['GP']
data['PPG'] = data['PTS'] / data['GP']
data['RPG'] = data['REB'] / data['GP']
data['APG'] = data['AST'] / data['GP']

In [17]:
data.iloc[0]

PLAYER_ID                   920
PLAYER_NAME          A.C. Green
NICKNAME                   A.C.
TEAM_ID              1610612747
TEAM_ABBREVIATION           LAL
                        ...    
SEASON                        0
MPG                   23.539045
PPG                    5.036585
RPG                    5.926829
APG                     0.97561
Name: 0, Length: 71, dtype: object

In [35]:
data.sort_values(['PLAYER_ID','AGE'])

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SEASON,MPG,PPG,RPG,APG
177,3,Grant Long,Grant,1610612763,VAN,34.0,42,13,29,0.310,...,313,257,213,23,264,0,21.958214,4.833333,5.571429,1.023810
611,3,Grant Long,Grant,1610612763,VAN,35.0,66,19,47,0.288,...,401,193,171,26,197,1,22.897601,6.000000,4.151515,1.257576
1047,3,Grant Long,Grant,1610612763,MEM,36.0,66,18,48,0.273,...,439,207,225,18,209,2,28.304571,6.318182,3.500000,2.060606
1479,3,Grant Long,Grant,1610612738,BOS,37.0,41,21,20,0.512,...,299,338,226,26,339,3,11.944593,1.756098,2.024390,0.609756
155,15,Eric Piatkowski,Eric,1610612746,LAC,29.0,75,13,62,0.173,...,432,185,213,23,174,0,22.815978,8.720000,2.960000,1.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11673,1641926,Dexter Dennis,Dexter,1610612742,DAL,25.0,4,1,3,0.250,...,283,517,257,38,514,24,7.466667,5.500000,2.250000,1.000000
11971,1641931,Onuralp Bitim,Onuralp,1610612741,CHI,25.0,23,11,12,0.478,...,383,442,257,38,437,24,11.651812,3.478261,1.391304,0.565217
11947,1641970,Mãozinha Pereira,Mãozinha,1610612763,MEM,23.0,7,2,5,0.286,...,216,450,190,38,447,24,17.376190,6.857143,5.285714,0.285714
12066,1641998,Trey Jemison,Trey,1610612763,MEM,24.0,25,5,20,0.200,...,470,345,190,38,353,24,22.959400,6.840000,5.360000,1.080000


In [43]:
# Count the number of instances for each PLAYER_ID
player_counts = data['PLAYER_ID'].count()

# # Convert the Series to a DataFrame
# player_counts_df = player_counts.reset_index()
# player_counts_df.columns = ['PLAYER_ID', 'Count']


In [41]:
type(data)

pandas.core.frame.DataFrame

In [45]:
# Group by PLAYER_ID and count the occurrences
player_counts_df = data.groupby('PLAYER_ID').size().reset_index(name='Count')


In [47]:
player_counts_df

,PLAYER_ID,Count
0,3,4
1,15,9
2,21,3
3,22,1
4,23,1
...,...,...
2441,1641926,1
2442,1641931,1
2443,1641970,1
2444,1641998,1


In [49]:
# # Group by PLAYER_ID and count the occurrences
# player_counts_df = df.groupby('PLAYER_ID').size().reset_index(name='Count')

# Filter for PLAYER_IDs with a count of 5 or more
filtered_player_counts_df = player_counts_df[player_counts_df['Count'] >= 5]


In [53]:
fdf=filtered_player_counts_df


In [63]:
fdata=data[data['PLAYER_ID'].isin(fdf['PLAYER_ID'])]

In [67]:
fdata.sort_values(['PLAYER_ID','AGE'])

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SEASON,MPG,PPG,RPG,APG
155,15,Eric Piatkowski,Eric,1610612746,LAC,29.0,75,13,62,0.173,...,432,185,213,23,174,0,22.815978,8.720000,2.960000,1.080000
589,15,Eric Piatkowski,Eric,1610612746,LAC,30.0,81,31,50,0.383,...,313,138,224,26,128,1,26.475576,10.617284,2.975309,1.185185
1025,15,Eric Piatkowski,Eric,1610612746,LAC,31.0,71,32,39,0.451,...,266,189,176,18,173,2,24.178216,8.816901,2.591549,1.577465
1458,15,Eric Piatkowski,Eric,1610612746,LAC,32.0,62,20,42,0.323,...,147,216,226,26,197,3,21.948683,9.693548,2.516129,1.129032
1886,15,Eric Piatkowski,Eric,1610612745,HOU,33.0,49,22,27,0.449,...,209,322,235,19,309,4,14.328231,4.102041,1.489796,0.530612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9543,1629750,Javonte Green,Javonte,1610612738,BOS,26.0,48,33,15,0.688,...,334,346,239,29,354,20,9.746840,3.395833,1.937500,0.541667
10084,1629750,Javonte Green,Javonte,1610612741,CHI,27.0,41,19,22,0.463,...,184,367,245,29,370,21,11.554268,3.560976,1.731707,0.414634
10651,1629750,Javonte Green,Javonte,1610612741,CHI,28.0,65,38,27,0.585,...,206,208,268,40,222,22,23.369923,7.200000,4.246154,0.923077
11218,1629750,Javonte Green,Javonte,1610612741,CHI,29.0,32,13,19,0.406,...,142,361,253,39,368,23,15.008021,5.156250,2.750000,0.718750


In [71]:
player_dfs = {player_id: group for player_id, group in fdata.groupby('PLAYER_ID')}

In [77]:
# Specify the columns to keep
columns_to_keep = ['AGE', 'GP', 'PPG','APG','RPG','MPG']  # Replace with your actual column names


In [79]:
# Create a new dictionary with filtered DataFrames
filtered_player_dfs = {player_id: group[columns_to_keep] for player_id, group in player_dfs.items()}


In [87]:
len(filtered_player_dfs)

991

In [89]:
# Remove entries with any NaN values
filtered_player_dfs_no_nan = {player_id: group for player_id, group in filtered_player_dfs.items() if not group.isna().any().any()}


In [91]:
len(filtered_player_dfs_no_nan)

991

In [93]:
filtered_player_dfs_no_nan[15]

,AGE,GP,PPG,APG,RPG,MPG
155,29.0,75,8.720000,1.080000,2.960000,22.815978
589,30.0,81,10.617284,1.185185,2.975309,26.475576
1025,31.0,71,8.816901,1.577465,2.591549,24.178216
1458,32.0,62,9.693548,1.129032,2.516129,21.948683
1886,33.0,49,4.102041,0.530612,1.489796,14.328231
2337,34.0,68,4.764706,0.750000,1.161765,12.340809
2809,35.0,29,2.034483,0.448276,0.793103,7.870000
3259,36.0,11,2.454545,0.363636,0.818182,6.618182
3721,37.0,16,2.437500,0.625000,0.750000,7.041667


In [95]:
# Truncate entries with more than 5 rows
truncated_player_dfs = {player_id: group.iloc[-5:] for player_id, group in filtered_player_dfs_no_nan.items() if len(group) > 5}


In [97]:
truncated_player_dfs[15]

,AGE,GP,PPG,APG,RPG,MPG
1886,33.0,49,4.102041,0.530612,1.489796,14.328231
2337,34.0,68,4.764706,0.750000,1.161765,12.340809
2809,35.0,29,2.034483,0.448276,0.793103,7.870000
3259,36.0,11,2.454545,0.363636,0.818182,6.618182
3721,37.0,16,2.437500,0.625000,0.750000,7.041667


In [99]:
tpdfs=truncated_player_dfs

In [113]:
list(tpdfs.keys())[0:5]

[15, 56, 57, 72, 84]

In [115]:
toX=[tpdfs[id][0:4]for id in list(tpdfs.keys())]

In [117]:
toX[0:5]

[       AGE  GP       PPG       APG       RPG        MPG
 1886  33.0  49  4.102041  0.530612  1.489796  14.328231
 2337  34.0  68  4.764706  0.750000  1.161765  12.340809
 2809  35.0  29  2.034483  0.448276  0.793103   7.870000
 3259  36.0  11  2.454545  0.363636  0.818182   6.618182,
        AGE  GP        PPG       APG       RPG        MPG
 1470  34.0  80  20.425000  8.287500  4.175000  40.120188
 1898  35.0  82  14.621951  5.475610  4.170732  34.585915
 2350  36.0  77  11.337662  6.090909  3.064935  33.125000
 2821  37.0  81   7.728395  3.172840  2.876543  28.455206,
        AGE  GP        PPG       APG       RPG        MPG
 1444  33.0  80   9.350000  4.700000  4.275000  33.870479
 1872  34.0  82  10.134146  4.231707  4.012195  33.937642
 2322  35.0  52   6.634615  3.826923  3.403846  29.237596
 2792  36.0   7   3.714286  2.000000  1.857143  26.456667,
        AGE  GP       PPG       APG       RPG        MPG
 677   30.0  33  7.454545  4.060606  2.212121  25.755707
 1119  31.0  76  9

In [131]:
toY=[tpdfs[id]['PPG'].iloc[4]for id in list(tpdfs.keys())]

In [133]:
toY[0:5]

[2.4375, 5.264705882352941, 1.8571428571428572, 4.72093023255814, 12.7625]

In [135]:
len(toX)

849

In [137]:
len(toY)

849

In [145]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

# Example data for 2 players over 5 seasons
# Step 1: Combine DataFrames into one with an identifier for each player
combined_data = []
for player_idx, df in enumerate(toX):
    df['Player'] = player_idx  # Add a player identifier
    combined_data.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(combined_data)

# Step 2: Prepare the input tensor
num_features = 6  # AGE, GP, PPG, APG, RPG, MPG
tensor_data = []

# Group by Player and extract the relevant features for each season
for player in combined_df['Player'].unique():
    player_data = combined_df[combined_df['Player'] == player][['AGE', 'GP', 'PPG', 'APG', 'RPG', 'MPG']]
    
    # Check if the player has enough seasons of data
    if len(player_data) >= 4:  # Adjust if you have more/less seasons
        # Convert the DataFrame to a numpy array and then to a tensor
        tensor_data.append(player_data.to_numpy())

x_temporal = torch.tensor([
    # Player 1
 tensor_data
], dtype=torch.float32)

# Target: Predict next season's PPG
toY2=[list(i) for i in toY]
y_target = torch.tensor([
    toY2  
], dtype=torch.float32)

# Scale the input data
scaler = MinMaxScaler()

# Reshape to 2D for scaling
x_reshaped = x_temporal.view(-1, 3).numpy()
x_scaled = scaler.fit_transform(x_reshaped)

# Reshape back to original shape
x_temporal_scaled = torch.tensor(x_scaled).view(x_temporal.shape).float()

# Scale the target (PPG values)
y_target_scaled = scaler.fit_transform(y_target)

# Model definition (Updated: No ReLU in final layer)
class SimpleTFT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.1):
        super(SimpleTFT, self).__init__()
        
        # LSTM to handle the temporal nature of the data
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to predict next PPG
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)
        
        # Take last timestep's output
        last_timestep = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_dim)
        
        # Fully connected output
        output = self.fc(last_timestep)
        return output

# Hyperparameters
input_dim = 6    # 3 features: PPG, Age, Games Played
hidden_dim = 16  # Number of hidden units
output_dim = 1   # Predicting 1 value: next season's PPG
num_layers = 2   # Number of LSTM layers

# Create the model
model = SimpleTFT(input_dim, hidden_dim, output_dim, num_layers)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adjusted learning rate

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero gradients
    
    # Forward pass
    output = model(x_temporal_scaled)
    
    # Compute loss
    loss = criterion(output, torch.tensor(y_target_scaled, dtype=torch.float32))
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predictions_scaled = model(x_temporal_scaled)
    
    # Inverse scale predictions back to original range
    predictions = scaler.inverse_transform(predictions_scaled)
    
    print("Predicted PPG for next season:")
    print(predictions)

/var/folders/c7/tjnc1nv13vl7684g7d2d8thm0000gn/T/ipykernel_72712/3751348251.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player'] = player_idx  # Add a player identifier


TypeError: 'numpy.float64' object is not iterable

In [147]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assume toX is a list of DataFrames for each player and toY is the target values

# Step 1: Combine DataFrames into one with an identifier for each player
combined_data = []
for player_idx, df in enumerate(toX):
    df.loc[:, 'Player'] = player_idx  # Add a player identifier using .loc
    combined_data.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(combined_data)

# Step 2: Prepare the input tensor
num_features = 6  # AGE, GP, PPG, APG, RPG, MPG
tensor_data = []

# Group by Player and extract the relevant features for each season
for player in combined_df['Player'].unique():
    player_data = combined_df[combined_df['Player'] == player][['AGE', 'GP', 'PPG', 'APG', 'RPG', 'MPG']]
    
    # Check if the player has enough seasons of data
    if len(player_data) >= 4:  # Adjust if you have more/less seasons
        # Convert the DataFrame to a numpy array
        tensor_data.append(player_data.to_numpy())

# Stack the player data into a tensor
x_temporal = torch.tensor(tensor_data, dtype=torch.float32)  # Shape: (num_players, num_seasons, num_features)

# Target: Predict next season's PPG
# Ensure toY is a 2D structure where each element corresponds to the player's target PPG for the next season
y_target = torch.tensor(toY, dtype=torch.float32).view(-1, 1)  # Reshape if necessary

# Scale the input data
scaler = MinMaxScaler()

# Reshape to 2D for scaling
x_reshaped = x_temporal.view(-1, num_features).numpy()
x_scaled = scaler.fit_transform(x_reshaped)

# Reshape back to original shape
x_temporal_scaled = torch.tensor(x_scaled).view(x_temporal.shape).float()

# Scale the target (PPG values)
y_target_scaled = scaler.fit_transform(y_target.numpy())

# Model definition (Updated: No ReLU in final layer)
class SimpleTFT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.1):
        super(SimpleTFT, self).__init__()
        
        # LSTM to handle the temporal nature of the data
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to predict next PPG
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)
        
        # Take last timestep's output
        last_timestep = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_dim)
        
        # Fully connected output
        output = self.fc(last_timestep)
        return output

# Hyperparameters
input_dim = num_features    # 6 features: AGE, GP, PPG, APG, RPG, MPG
hidden_dim = 16  # Number of hidden units
output_dim = 1   # Predicting 1 value: next season's PPG
num_layers = 2   # Number of LSTM layers

# Create the model
model = SimpleTFT(input_dim, hidden_dim, output_dim, num_layers)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adjusted learning rate

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero gradients
    
    # Forward pass
    output = model(x_temporal_scaled)
    
    # Compute loss
    loss = criterion(output, torch.tensor(y_target_scaled, dtype=torch.float32))
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predictions_scaled = model(x_temporal_scaled)
    
    # Inverse scale predictions back to original range
    predictions = scaler.inverse_transform(predictions_scaled)
    
    print("Predicted PPG for next season:")
    print(predictions)


Epoch [10/100], Loss: 0.0244
Epoch [20/100], Loss: 0.0229
Epoch [30/100], Loss: 0.0210
Epoch [40/100], Loss: 0.0166
Epoch [50/100], Loss: 0.0101
Epoch [60/100], Loss: 0.0090
Epoch [70/100], Loss: 0.0077
Epoch [80/100], Loss: 0.0066
Epoch [90/100], Loss: 0.0059
Epoch [100/100], Loss: 0.0052
Predicted PPG for next season:
[[ 1.41708403]
 [ 4.42023765]
 [ 3.22261888]
 [ 3.4546658 ]
 [12.37276923]
 [ 1.4489957 ]
 [ 6.99484744]
 [ 2.30947408]
 [ 3.73977408]
 [ 6.0174311 ]
 [ 6.91350963]
 [ 3.85192555]
 [10.49043855]
 [10.81008361]
 [ 8.99269496]
 [ 4.50249564]
 [ 3.15518305]
 [ 5.83623917]
 [ 1.48500787]
 [ 1.82530323]
 [ 2.61760809]
 [ 6.06678692]
 [ 3.62973387]
 [17.75478841]
 [ 1.66063942]
 [ 2.11072324]
 [ 0.94387109]
 [ 4.10076939]
 [15.38275804]
 [ 3.9598876 ]
 [ 2.09601069]
 [ 1.51644652]
 [ 2.91937889]
 [ 3.91163599]
 [ 4.01970454]
 [ 3.30216802]
 [ 4.27563171]
 [ 3.95296246]
 [ 3.8360488 ]
 [ 3.63614825]
 [ 6.5956336 ]
 [ 6.68468645]
 [ 1.48814268]
 [ 4.246402  ]
 [ 5.16752808]
 [ 

In [167]:
print([round(float(i),2) for i in predictions[40:50]])
print([round(j,2) for j in toY[40:50]])

[6.6, 6.68, 1.49, 4.25, 5.17, 4.37, 1.28, 2.9, 4.0, 2.72]
[14.76, 9.22, 3.0, 3.44, 6.44, 4.29, 2.0, 3.38, 3.16, 6.03]


/var/folders/c7/tjnc1nv13vl7684g7d2d8thm0000gn/T/ipykernel_72712/3502679692.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print([round(float(i),2) for i in predictions[40:50]])


In [178]:
preds= [round(float(i),2) for i in predictions]
targs = [round(j,2) for j in toY]

/var/folders/c7/tjnc1nv13vl7684g7d2d8thm0000gn/T/ipykernel_72712/1692988623.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  preds= [round(float(i),2) for i in predictions]


In [182]:
print(preds[0:10])
print(targs[0:10])

[1.42, 4.42, 3.22, 3.45, 12.37, 1.45, 6.99, 2.31, 3.74, 6.02]
[2.44, 5.26, 1.86, 4.72, 12.76, 1.78, 5.46, 2.49, 6.28, 2.27]


In [188]:
errors = [i / j if j != 0 else float('inf') for i, j in zip(preds, targs)]

In [190]:
errors[0:20]

[0.5819672131147541,
 0.8403041825095057,
 1.7311827956989247,
 0.7309322033898306,
 0.9694357366771159,
 0.8146067415730337,
 1.2802197802197803,
 0.927710843373494,
 0.5955414012738853,
 2.6519823788546253,
 3.30622009569378,
 1.774193548387097,
 2.8198924731182795,
 2.7789203084832907,
 0.8026785714285715,
 0.9164969450101833,
 1.025974025974026,
 1.5614973262032084,
 0.9371069182389936,
 0.8318181818181818]

In [194]:
numA=0
mostlyAcc=[numA+=1 if abs(1-x)<.2 for x in errors]

SyntaxError: invalid syntax (4117178732.py, line 2)

In [200]:
numA=0
for x in errors:
    if abs(1-x)<.2:
        numA+=1
print(numA)        

286


In [212]:
numA=0 # Accurate
numVA=0 # Very Accurate
numXA=0
numLA=0
numNot=0
for x in errors:
    if abs(1-x)<.05:
        numXA+=1
    elif abs(1-x)<.1:
        numVA+=1
    elif abs(1-x)<.2:
        numA+=1
    elif abs(1-x)<.3:
        numLA+=1
    else:
        numNot+=1
print(numXA)
print(numVA)
print(numA)
print(numLA)
print(numNot)

82
69
135
126
437
